## Create a function to upload files from s3 storage link

In [148]:
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd

- I would like to fetch and extract data files from the provided html link using requesets and beautifulsoup
- This will help the flow in the future unless partitions have changed

In [465]:
html_url = "https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/data-modeling.html"

r = requests.get(html_url)
soup = BeautifulSoup(r.content, 'html.parser')

In [466]:
parsed_html = soup

In [467]:
# found the file as a list we have about 3 files
amount_of_files = len(parsed_html.find_all('a'))

In [632]:
# looking how to parse individually for file name
data_files = parsed_html.find_all('a')[0]['href'].split('/')[-1].split('.')[0]
data_files

'receipts'

In [634]:
# parsing the actual link to upload the file
file_urls = parsed_html.find_all('a')[1]['href']
file_urls

'https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/users.json.gz'

In [616]:
data_files

'receipts'

In [636]:
def parseHtml(html_url):
    
    r = requests.get(html_url)
    soup = BeautifulSoup(r.content, 'html.parser')
    parsed_html = soup
    amount_of_files = len(parsed_html.find_all('a'))
    data_names = []
    url_list = []
    for i in range(amount_of_files):
        data_files = parsed_html.find_all('a')[i]['href'].split('/')[-1].split('.')[0]
        file_urls = parsed_html.find_all('a')[i]['href']
        
        data_names.append(data_files)
        url_list.append(file_urls)
    
    return data_names, url_list

In [644]:
dataNames, urlList = parseHtml(html_url)
print('names:', dataNames)
print('urls:', urlList)

names: ['receipts', 'users', 'brands']
urls: ['https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/receipts.json.gz', 'https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/users.json.gz', 'https://fetch-hiring.s3.amazonaws.com/data-analyst/ineeddata-data-modeling/brands.json.gz']


- After scraping the HTML I found that it was coming in as bytes
- We need to download the json.gz file and open it as a json file to read from pandas
- We also want delete the file after we turn it into a json file (no need for extra files)
- Last would be to get the json read as a pandas dataframe and save it as a csv file for further cleaning.

In [602]:
import wget
import tarfile
import shutil
import gzip
import os

# Code Breakdown (Scroll Down to run function)

In [628]:
# download the json.gz file
filename = wget.download(file_urls)

# unzip the gz file and write a new file as json
with gzip.open(filename) as g:
    with open(f'{data_files}.json', 'wb') as f_out:
        shutil.copyfileobj(g, f_out)

# if the gz file exists delete (save space) - or log for upload
if os.path.exists(f"{filename}"):
    os.remove(f"{filename}")
    print("old file removed ;)")
else:
    print("The file does not exist")

# if the file exist already we will rewrite it or we can log as new file if there is new date 
if os.path.exists(f"json_files/{data_files}.json"):
    shutil.copy(f'{data_files}.json', 'json_files/')
    print('File rewritten')
else:
# else the file does not exists we will move the file to the proper folder
    shutil.move(f'{data_files}.json', 'json_files/')
    print('File moved')


df = pd.read_json(f'json_files/{data_files}.json', lines = True)

path = 'csv_files/'
if not os.path.exists(path):
    os.makedirs(path)
    
df.to_csv(f'{path}{data_files}.csv')

old file removed ;)
File rewritten


In [683]:
def createFiles(data_files, file_urls): 
    
    for dfnames in data_files:
        
        for dfurls in file_urls:
        
            # download the json.gz file
            filename = wget.download(dfurls)
            names = dfnames
            # unzip the gz file and write a new file as json
            with gzip.open(filename) as g:
                with open(f'{names}.json', 'wb') as f_out:
                    shutil.copyfileobj(g, f_out)

            # if the gz file exists delete (save space) - or log for upload
            if os.path.exists(f"{filename}"):
                os.remove(f"{filename}")
                print("old file removed ;)")
            else:
                print("The file does not exist")

            # if the file exist already we will rewrite over to json file
            #  or we can log as new file if there is new date
            
            json_path = 'json_files/'
            if not os.path.exists(json_path):
                os.makedirs(json_path)
            
            if os.path.exists(f"json_files/{names}.json"):
                shutil.copy(f'{names}.json', 'json_files/')
                print('File rewritten')
            else:
            # else the file does not exists we will move the file to the proper folder
                shutil.move(f'{names}.json', 'json_files/')
                print('File moved')


            df = pd.read_json(f'json_files/{names}.json', lines = True)

            # create folder for csv_files
            csv_path = 'csv_files/'
            if not os.path.exists(csv_path):
                os.makedirs(csv_path)

            df.to_csv(f'{csv_path}{names}.csv')
            
            # remove any extra files lingering
            if os.path.exists(f"{names}.json"):
                os.remove(f"{names}.json")

    print("All files are uploaded")

- Had an issue here seems like it is iterating a few times and a Typeerror io.'_io.BufferedWriter
  - cou

In [684]:
createFiles(dataNames, urlList)

old file removed ;)
File rewritten
old file removed ;)
File rewritten
old file removed ;)
File rewritten
old file removed ;)
File rewritten
old file removed ;)
File rewritten
old file removed ;)
File rewritten
old file removed ;)
File rewritten
old file removed ;)
File rewritten
old file removed ;)
File rewritten
All files are uploaded


In [686]:
 df = pd.read_json(f'json_files/{data_files}.json', lines = True)

In [687]:
df

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...
1162,{'$oid': '5f77274dbe37ce6b592e90c0'},511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN
1163,{'$oid': '5dc1fca91dda2c0ad7da64ae'},511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL
1164,{'$oid': '5f494c6e04db711dd8fe87e7'},511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217
1165,{'$oid': '5a021611e4b00efe02b02a57'},511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves
